<a href="https://colab.research.google.com/github/kimheeseo/pytorch-RNN/blob/main/RNN_%EC%97%B0%EC%8A%B5_2021_12_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#출처 : https://justkode.kr/deep-learning/pytorch-rnn
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

"""
Word Processing
"""
word_list = list(set(" ".join(sentences)))
print('word_list값',word_list)


word_list값 ['c', 'd', 'o', 'i', 'g', 'm', ' ', 'y', 'v', 'f', 'k', 'a', 't', 'e', 'u', 'h', 'l']


In [78]:
word_list = list(set(" ".join(sentences).split()))
print(word_list)
word_dict = {w: i for i, w in enumerate(word_list)}
print('word_dict값',word_dict)
number_dict = {i: w for i, w in enumerate(word_list)}
print('number_dict값',number_dict)
n_class = len(word_dict)
print('n_class값',n_class)

['like', 'dog', 'i', 'you', 'coffee', 'cat', 'love', 'hate', 'milk']
word_dict값 {'like': 0, 'dog': 1, 'i': 2, 'you': 3, 'coffee': 4, 'cat': 5, 'love': 6, 'hate': 7, 'milk': 8}
number_dict값 {0: 'like', 1: 'dog', 2: 'i', 3: 'you', 4: 'coffee', 5: 'cat', 6: 'love', 7: 'hate', 8: 'milk'}
n_class값 9


In [79]:
"""
TextRNN Parameter
"""
batch_size = len(sentences)
print('batch_size값',batch_size)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈
print('sentences값',sentences)

inputtest_batch=[]
targettest_batch=[]
i=0
#다음 for문은 make_batch문의 동작을 보기위해서, 나눠서 표현방법을 본다.
for sen in sentences:
  print(i)
  i=i+1
  word=sen.split()
  print('word값',word)
  input=[word_dict[n] for n in word[:]]
  print('input값',input)
  target=word_dict[word[-1]]
  print('word[-1]값',word[-1])
  #print('np.eye(n_class)값',np.eye(n_class))
  #print('np.eye(n_class)[input]',np.eye(n_class)[input])
  inputtest_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
  #print('inputtest_batch값',inputtest_batch)
  print('target값',target)
  targettest_batch.append(target)

print('inputtest_batch값',inputtest_batch)
print('targettest_batch값',targettest_batch)

#split에 의해 1번째 for문 동작 : 'I like dog'이 sen값이 된다. 그리고 "word=sen.split()"에 의해서
#word값은 ['I','like']가 되고, input값은 [2,0]이 된다. 우선 2개의 값이 나오는 이유는 [:-1]에 의해서 그런것이고,
#만일 [:]이면, ['I','like','dog']가 되고 input값은 [2,0,1]이 되고, 이때 숫자는 앞서 word_dict을 통해 설정한
#I는 2, like는 0, dog는 1을 나타내기 때문에, 이와 같은 결과값이 나온다.

batch_size값 6
sentences값 ['i like dog', 'i love coffee', 'i hate milk', 'you like cat', 'you love milk', 'you hate coffee']
0
word값 ['i', 'like', 'dog']
input값 [2, 0, 1]
word[-1]값 dog
target값 1
1
word값 ['i', 'love', 'coffee']
input값 [2, 6, 4]
word[-1]값 coffee
target값 4
2
word값 ['i', 'hate', 'milk']
input값 [2, 7, 8]
word[-1]값 milk
target값 8
3
word값 ['you', 'like', 'cat']
input값 [3, 0, 5]
word[-1]값 cat
target값 5
4
word값 ['you', 'love', 'milk']
input값 [3, 6, 8]
word[-1]값 milk
target값 8
5
word값 ['you', 'hate', 'coffee']
input값 [3, 7, 4]
word[-1]값 coffee
target값 4
inputtest_batch값 [array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]]), array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]]), array

In [80]:
def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch  

In [86]:
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)
print('input_batch값',input_batch)
print('target_batch값',target_batch)

input_batch값 tensor([[[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]]], requires_grad=True)
target_batch값 tensor([1, 4, 8, 5, 8, 4])


In [82]:
"""
TextRNN
"""
class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN, self).__init__()

    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    print('model값',model)
    return model
	
"""
Training
"""
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]
print('input값',input)

model값 tensor([[ 2.2814, -1.3653,  1.1173, -2.0937, -1.2090, -2.3969,  0.7136, -0.8873,
         -2.4639],
        [ 1.6166, -0.6788,  0.9798, -2.1225, -1.7216, -2.1396,  0.2085, -0.9754,
         -1.6933],
        [ 1.1981, -0.9043,  0.2928, -1.2251, -0.6726, -2.6167,  0.0374, -1.1814,
         -1.7064],
        [ 2.0898, -0.9956,  0.7520, -1.6743, -0.8500, -2.5033,  0.6499, -0.7952,
         -1.9668],
        [ 1.3964, -0.2490,  0.6212, -1.7067, -1.3671, -2.2045,  0.1437, -0.8693,
         -1.1434],
        [ 0.9268, -0.5310, -0.1296, -0.7425, -0.2616, -2.7342, -0.0675, -1.1099,
         -1.1925]], grad_fn=<AddBackward0>)
model값 tensor([[ 2.1883, -1.1554,  0.9929, -2.0063, -1.0374, -2.3330,  0.6957, -0.8635,
         -2.2423],
        [ 1.5105, -0.4650,  0.8654, -1.9955, -1.4595, -2.0496,  0.2342, -0.9298,
         -1.4798],
        [ 1.0752, -0.7139,  0.1584, -1.0745, -0.4010, -2.5522,  0.0472, -1.1457,
         -1.5038],
        [ 1.9945, -0.7756,  0.6187, -1.5757, -0.6825, -2.4476

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [-1.7679,  1.5621, -2.1611, -0.6883,  4.2778,  0.6633, -0.8424, -1.5782,
          3.8494]], grad_fn=<AddBackward0>)
model값 tensor([[-0.1936,  3.2726, -0.1466, -2.4911,  0.9131,  3.0632,  0.1237, -0.7875,
          1.9482],
        [-1.6347,  1.9366, -2.3307, -0.6600,  4.3667,  0.7275, -0.8279, -1.4814,
          4.3288],
        [-1.5138,  1.6931, -2.6201, -0.4206,  4.6621,  0.1946, -0.8680, -1.5212,
          4.4355],
        [-0.3474,  2.8599,  0.4755, -2.8187, -0.0202,  3.2380,  0.1096, -0.8295,
          0.8345],
        [-1.8655,  1.7778, -1.8395, -0.8836,  3.0843,  0.6805, -1.0405, -1.4462,
          3.6929],
        [-1.7743,  1.5477, -2.1659, -0.6956,  4.2793,  0.6384, -0.8570, -1.5912,
          3.8349]], grad_fn=<AddBackward0>)
model값 tensor([[-0.1859,  3.2910, -0.1620, -2.4968,  0.9098,  3.0621,  0.1125, -0.7978,
          1.9398],
        [-1.6372,  1.9356, -2.3513, -0.6600,  4.3914,  0.7013, -0.8418, -1.4922,
          4.3433],


In [83]:
hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
print('hidden값',hidden)

predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print('predict값',predict)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

hidden값 tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], requires_grad=True)
model값 tensor([[ 9.1694e-01,  6.7056e+00, -1.9754e+00, -2.2107e+00, -1.1874e+00,
          1.3482e+00, -6.0144e-01, -1.0290e+00,  1.4807e+00],
        [-2.5606e+00,  1.3217e+00, -2.9841e+00, -1.8743e+00,  9.0569e+00,
         -6.0743e-01, -1.5225e+00, -2.6403e+00,  5.1003e+00],
        [ 1.4040e-01,  2.9959e+00, -2.9733e+00, -3.4020e-01,  3.7623e+00,
         -6.0572e+00, -9.6546e-01, -1.5288e+00,  8.0049e+00],
        [-8.5370e-01,  1.0547e+00,  6.1417e-02, -3.4892e+00, -7.1006e-03,
          6.2994e+00, -7.1255e-01, -2.0608e+00, -6.7673e+00],
        [-2.7617e+00, -7.8144e-01, -1.7090e+00, -1.8070e+00,  1.2319e+00,
         -1.8783e+00, -2.3425e+00, -2.0220e+00,  5.8194e+00],
        [-2.1343e+00, -1.3038e+00, -1.4799e+00, -1.9076e+00,  7.6890e+00,
          5.6775e-02, -

In [84]:
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print(number_dict[n.item()] for n in predict.squeeze())
print(predict.squeeze())

model값 tensor([[ 9.1694e-01,  6.7056e+00, -1.9754e+00, -2.2107e+00, -1.1874e+00,
          1.3482e+00, -6.0144e-01, -1.0290e+00,  1.4807e+00],
        [-2.5606e+00,  1.3217e+00, -2.9841e+00, -1.8743e+00,  9.0569e+00,
         -6.0743e-01, -1.5225e+00, -2.6403e+00,  5.1003e+00],
        [ 1.4040e-01,  2.9959e+00, -2.9733e+00, -3.4020e-01,  3.7623e+00,
         -6.0572e+00, -9.6546e-01, -1.5288e+00,  8.0049e+00],
        [-8.5370e-01,  1.0547e+00,  6.1417e-02, -3.4892e+00, -7.1006e-03,
          6.2994e+00, -7.1255e-01, -2.0608e+00, -6.7673e+00],
        [-2.7617e+00, -7.8144e-01, -1.7090e+00, -1.8070e+00,  1.2319e+00,
         -1.8783e+00, -2.3425e+00, -2.0220e+00,  5.8194e+00],
        [-2.1343e+00, -1.3038e+00, -1.4799e+00, -1.9076e+00,  7.6890e+00,
          5.6775e-02, -1.1546e+00, -2.8629e+00,  8.0363e-01]],
       grad_fn=<AddBackward0>)
<generator object <genexpr> at 0x7f57766e8b50>
tensor([1, 4, 8, 5, 8, 4])
